In [4]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_data}sumstat"
PATH_SUMSTAT_no_corr = f"{PATH_data}sumstat_no_corr"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"


PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'
SEABORN_COLORS = [
    'black',
    'k',
    'dimgray',
    'dimgrey',
    'gray',
    'grey',
    'lightgray',
    'lightgrey',
    'darkgrey',
    'silver',
    'lightgray',
    'lightgrey',
    'gainsboro',
    'whitesmoke',
    'w',
    'white',
    'snow',
    'rosybrown',
    'lightcoral',
    'indianred',
    'brown',
    'firebrick',
    'maroon',
    'darkred',
    'r',
    'red',
    'mistyrose',
    'salmon',
    'tomato',
    'darksalmon',
    'coral',
    'orangered',
    'lightsalmon',
    'sienna',
    'seashell',
    'chocolate',
    'saddlebrown',
    'sandybrown',
    'peachpuff',
    'peru',
    'linen',
    'bisque',
    'darkorange',
    'burlywood',
    'antiquewhite',
    'tan',
    'navajowhite',
    'blanchedalmond',
    'papayawhip',
    'moccasin',
    'orange',
    'wheat',
    'oldlace',
    'floralwhite',
    'darkgoldenrod',
    'goldenrod',
    'cornsilk',
    'gold',
    'lemonchiffon',
    'khaki',
    'palegoldenrod',
    'darkkhaki',
    'ivory',
    'beige',
    'lightyellow',
    'lightgoldenrodyellow',
    'olive',
    'y',
    'yellow',
    'olivedrab',
    'yellowgreen',
    'darkolivegreen',
    'greenyellow',
    'chartreuse',
    'lawngreen',
    'honeydew',
    'darkseagreen',
    'palegreen',
    'lightgreen',
    'forestgreen',
    'limegreen',
    'darkgreen',
    'g',
    'green',
    'lime',
    'seagreen',
    'mediumseagreen',
    'springgreen',
    'mintcream',
    'mediumspringgreen',
    'mediumaquamarine',
    'aquamarine',
    'turquoise',
    'lightseagreen',
    'mediumturquoise',
    'azure',
    'lightcyan',
    'paleturquoise',
    'darkslategray',
    'darkslategrey',
    'teal',
    'darkcyan',
    'c',
    'aqua',
    'cyan',
    'darkturquoise',
    'cadetblue',
    'powderblue',
    'lightblue',
    'deepskyblue',
    'skyblue',
    'lightskyblue',
    'steelblue',
    'aliceblue',
    'dodgerblue',
    'lightslategray',
    'lightslategrey',
    'slategrey',
    'slategray',
    'lightsteelblue',
    'cornflowerblue',
    'royalblue',
    'ghostwhite',
    'lavender',
    'midnightblue',
    'navy',
    'darkblue',
    'mediumblue',
    'b',
    'blue',
    'slateblue',
    'darkslateblue',
    'mediumslateblue',
    'mediumpurple',
    'rebeccapurple',
    'blueviolet',
    'indigo',
    'darkorchid',
    'darkviolet',
    'mediumorchid',
    'thistle',
    'plum',
    'violet',
    'purple',
    'darkmagenta',
    'm',
    'fuchsia',
    'magenta',
    'orchid',
    'mediumvioletred',
    'deeppink',
    'hotpink',
    'lavenderblush',
    'palevioletred',
    'crimson',
    'pink',
    'lightpink'
]

In [5]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from abyss.helpers import metrics_viewer
from abyss.glob import glob
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

ModuleNotFoundError: No module named 'matplotlib'